# Data recovering

This first notebook aims at retrieving a minimal set of data for running further analysis.

**Warning:** In these notebooks, we will interact with a database, by adding fresh data. As a wrong move is always possible, an act-first-and-think-then tool that erase data folder and reset the database is provided (see `./resetdb.sh`). Use it with caution! :)

## Introduction

In [ ]:
from datetime import datetime, date, timedelta
import json
import os
import subprocess
import zipfile

In [ ]:
import fiona
import folium
import pandas as pd
import requests
import shapely.geometry
from sqlalchemy import create_engine

## Configuration

In [ ]:
DATADIR = "../data"

In [ ]:
os.makedirs(os.path.join(DATADIR, "lyon"), exist_ok=True)

In [ ]:
HOST = "localhost"
PORT = 5432
USER = "rde"
DBNAME = "jitenshea"

## Utilities

In [ ]:
def get_engine():
    url = "postgresql://{user}@{host}:{port}/{dbname}".format(user=USER, host=HOST, port=PORT, dbname=DBNAME)
    return create_engine(url)

In [ ]:
def create_schema(schema):
    engine = get_engine()
    engine.execute("CREATE SCHEMA IF NOT EXISTS {schema};".format(schema=schema))

Be careful to create the `lyon` schema at this step, otherwise further ones will fail...

In [ ]:
engine = get_engine()

In [ ]:
create_schema("lyon")

## Retrieve the data

### Download the raw station data

We download the station information, and save the resulting archive into the data folder:

In [ ]:
archive_path = os.path.join(DATADIR, "lyon", "lyon-stations.zip")

In [ ]:
LYON_SRID = 3946
LYON_STATION_URL = "https://download.data.grandlyon.com/wfs/grandlyon?SERVICE=WFS&VERSION=2.0.0&outputformat=SHAPEZIP&request=GetFeature&SRSNAME=EPSG:3946&typename=pvo_patrimoine_voirie.pvostationvelov"

In [ ]:
with open(archive_path, "wb") as fobj:
    resp = requests.get(LYON_STATION_URL)
    resp.raise_for_status()
    fobj.write(resp.content)

### Download bike availability history

#### Method 1: Latest records + Cron job

In [ ]:
LYON_REALTIME_URL = "https://download.data.grandlyon.com/ws/rdata/jcd_jcdecaux.jcdvelov/all.json"

In [ ]:
timestamp = datetime.now()
realtime_json_file = timestamp.strftime("%HH%MM")
realtime_json_path = os.path.join(DATADIR, "lyon", str(timestamp.year), str(timestamp.month), str(timestamp.day), realtime_json_file + ".json")
os.makedirs(os.path.dirname(realtime_json_path), exist_ok=True)
print(realtime_json_path)

session = requests.Session()
resp = session.get(LYON_REALTIME_URL)
with open(realtime_json_path, "w") as fobj:
    json.dump(resp.json(), fobj, ensure_ascii=False)
session.close()


This method provides the freshest bike availability data, hence one could build a really big history by repeating the dump each X minutes (X being a frequency of your choice...).

By setting a cron job, this task could be done properly. In your shell:
```
crontab -e
```
Then in the crontab file:
```
# m h  dom mon dow   command
*/5 * * * * the-program
```
This last example would execute `the-program` each 5 minutes, every hour of every day of every month... However it is beyond the scope of this modest workshop!

*NOTE:* This is still the best way to get bike-sharing system data, anyway!

#### Method 2: It's your birthday!

No need to mess up the cron jobs on your laptop in a quick-and-dirt move, we are lucky! Some investigations on Lyon open data portal give us a ready-to-exploit toy dataset: the 7 last days of bike availability, measured every 5 minutes *(sounds perfect, isn't it?)*:

https://download.data.grandlyon.com/catalogue/srv/eng/catalog.search#/metadata/9bc6806d-e8a0-463b-aaa1-4364a75e44d7

In [ ]:
LYON_AVAILABILITY_URL = "https://download.data.grandlyon.com/sos/velov?request=GetObservation&service=SOS&version=1.0.0&offering=reseau_velov&observedProperty=bikes&eventTime={begin}/{end}&responseFormat=application/json"

Before to retrieve the raw history data, we need some piping miscellanea:

In [ ]:
def one_week_before(timestamp):
    return timestamp - timedelta(7)

In [ ]:
stop = date.today()
start = one_week_before(stop)

In [ ]:
start_date = start.strftime("%Y-%m-%dT%H:%M:%SZ")
stop_date = stop.strftime("%Y-%m-%dT%H:%M:%SZ")
LYON_AVAILABILITY_FULL_URL = LYON_AVAILABILITY_URL.format(begin=start_date, end=stop_date)
LYON_AVAILABILITY_FULL_URL

In [ ]:
availability_output_file = "{begin}-{end}.json".format(begin=start.strftime("%Y%m%d"), end=stop.strftime("%Y%m%d"))
availability_output_path = os.path.join(DATADIR, "lyon", "history", availability_output_file)
os.makedirs(os.path.dirname(availability_output_path), exist_ok=True)
availability_output_path

Here we have defined the final download URL, as well as an output path on the file system, we can do the job:

In [ ]:
session = requests.Session()
resp = session.get(LYON_AVAILABILITY_FULL_URL)
with open(availability_output_path, "w") as fobj:
    json.dump(resp.json(), fobj, ensure_ascii=False)

In [ ]:
ls ../data/lyon/history

The bike availability (recent) history is on our computers!

## Store the data into the database

### Unzip the downloaded station archive

Once we have got the archive file, we may unzip it and retrieve the Lyon stations as shapefiles:

In [ ]:
zip_ref = zipfile.ZipFile(archive_path)
zip_ref.extractall(os.path.dirname(archive_path))
zip_ref.close()

### Store the raw station data into the database

In [ ]:
LYON_DATANAME = "pvo_patrimoine_voirie.pvostationvelov"

In [ ]:
shp_file = os.path.join(os.path.dirname(archive_path), LYON_DATANAME + ".shp")

#### The loat solution: shell command pipe with `sh`

Here the station information lies into the shapefiles, we still have to store it into the application database. We use `shp2pgsql` and `psql` for this purpose.

In [ ]:
cmd = "shp2pgsql -s 4326 " + shp_file + " lyon.raw_station"
cmd += " | psql -h " + HOST + " -d " + DBNAME + " -U " + USER + " -p " + str(PORT)
print(cmd)
subprocess.call(cmd, shell=True)

#### Alternative solution : pure Python with `fiona` + `SQLAlchemy`

In [ ]:
velov = fiona.open(shp_file)

In [ ]:
velov.schema["properties"]

In [ ]:
params_str = ""
for key, value in velov.schema["properties"].items():
    vtype = "character varying(80)" if value == "str:80" else value
    params_str += key + " " + vtype + ","
srid = velov.crs["init"].split(":")[1]
params_str += f"geom geometry(Point, {srid})"
engine.execute("DROP TABLE IF EXISTS lyon.raw_station;")
engine.execute("CREATE TABLE lyon.raw_station ({});".format(params_str))

In [ ]:
for station in velov:
    values = tuple(station["properties"].values())
    point = shapely.geometry.Point(station["geometry"]["coordinates"])
    query = "INSERT INTO lyon.raw_station VALUES (" + ", ".join(["%s"] * len(values)) + f", ST_SETSRID(%s::geometry, {srid}))"
    engine.execute(query, *values, point.wkt)

Now the station should be in the database in a raw format. We can check it:

In [ ]:
engine = get_engine()
rset = engine.execute("SELECT count(*) FROM lyon.raw_station;")
rset.fetchone()

In [ ]:
engine.execute("SELECT gid, st_x(geom), st_y(geom) FROM lyon.raw_station").fetchall()[:4]

In [ ]:
engine.execute("SELECT st_srid(geom) FROM lyon.raw_station").fetchone()

There is 369 bike-sharing stations in Lyon!

### Consider a standardized version of station data

At this point, one could stop the station data treatment. However we can still improve the design of our data; especially if we target to retrieve data in additional cities.

Here we will "simply" build a new table with fixed attributes. A particular attention must be paid on raw attributes (typically they can be known after exploring the data itself).

In [ ]:
query = """
DROP TABLE IF EXISTS lyon.station;
CREATE TABLE lyon.station(
id varchar,
name varchar(250),
address varchar(250),
city varchar(100),
nb_stands int,
geom geometry(POINT, 4326)
);
INSERT INTO lyon.station
SELECT {id} AS id,
{name} AS name,
{address} AS address,
{city} AS city,
{nb_stands}::int AS nb_stands,
ST_TRANSFORM(ST_FORCE2D(geom), 4326) AS geom
FROM lyon.raw_station
"""

In [ ]:
LYON_ID = "idstation"
LYON_NAME = "nom"
LYON_ADDRESS = "adresse1"
LYON_CITY = "commune"
LYON_NB_STATIONS = "nbbornette"
engine.execute(query.format(id=LYON_ID, name=LYON_NAME, address=LYON_ADDRESS, city=LYON_CITY, nb_stands=LYON_NB_STATIONS))
rset = engine.execute("SELECT count(*) FROM lyon.station;")
rset.fetchone()

In [ ]:
rset = engine.execute("SELECT id, name, st_x(geom), st_y(geom) FROM lyon.station").fetchall()
rset[:4]

### Let's plot it with `folium`

In [ ]:
stations = pd.DataFrame(rset, columns=["id", "name", "x", "y"])

In [ ]:
def generateBaseMap(default_location, default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
m = generateBaseMap([stations.y.mean(), stations.x.mean()])
for station in stations.values:
    folium.CircleMarker(
        location=(station[3], station[2]), radius=3, fill=True, popup=": ".join([station[0], station[1]])
    ).add_to(m)
m

### Store the bike availability history into a csv file

Let come back to the bike availability data. We downloaded it in the `json` format, however a more convenient format is the `csv`: as tables, the data could be far easier to handle, and to store into the application database.

In [ ]:
def convert_history_data(history_file):
    """Read the bike availability history data, and send it directly into a csv file
    
    The function, and especially the json file structure, is infered from the Lyon Open Data portal.
    """
    with open(history_file, "r") as fobj:
        data = json.load(fobj)
        datalist = []
        for d in data["ObservationCollection"]["member"]:
            cur_d = d["result"]["DataArray"]["values"]
            station_id = d["name"].split("-")[1]
            cur_d = [
                [item[0], int(float(item[1])), station_id]
                for item in cur_d
            ]
            datalist += cur_d
        df = pd.DataFrame(
            datalist, columns=["timestamp", "available_bikes", "id"]
        )
        df.loc[:, "timestamp"] = pd.to_datetime(df["timestamp"])
        df.sort_values("timestamp")
        with open(history_file.replace(".json", ".csv"), "w") as fobj:
            df.to_csv(fobj, index=False)

In [ ]:
convert_history_data(availability_output_path)

In [ ]:
ls ../data/lyon/history

Now it should be easier to populate the database...

### Store the bike availibility history into the database

In [ ]:
availability_timeseries = pd.read_csv(availability_output_path.replace(".json", ".csv"), parse_dates=["timestamp"])

In [ ]:
availability_timeseries.info()

For a sake of data consistency, store the IDs as strings (similarly to `lyon.station` table):

In [ ]:
availability_timeseries.loc[:, "id"] = availability_timeseries["id"].astype(str)

In [ ]:
availability_timeseries.head()

In [ ]:
availability_timeseries.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(style="whitegrid")

In [ ]:
def get_station_ts(station_id):
    return availability_timeseries.loc[availability_timeseries["id"] == station_id].drop(columns=["id"]).set_index("timestamp")
f, ax = plt.subplots(3, 1, figsize=(15, 18))
ax[0].set_title("station 1001")
sns.lineplot(data=get_station_ts("1001"), ax=ax[0])
ax[1].set_title("station 3080")
sns.lineplot(data=get_station_ts("3080"), ax=ax[1])
ax[2].set_title("station 10116")
sns.lineplot(data=get_station_ts("10116"), ax=ax[2])

In [ ]:
engine.execute("DROP TABLE IF EXISTS lyon.timeseries;")
availability_timeseries.to_sql("timeseries", schema="lyon", con=engine, chunksize=50000, method="multi", index=False)

In [ ]:
rset = engine.execute("SELECT count(*) FROM lyon.timeseries;")
rset.fetchone()

As this point, we have built our database, and populated it with station and bike availability data!